In [89]:
import csv
import random
from sklearn import naive_bayes
from sklearn.ensemble.forest import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.tree.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model.ridge import Ridge
import sklearn.metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors.regression import KNeighborsRegressor
import pandas as pd

import numpy as np
from sklearn import cross_validation
from sklearn import datasets

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.cross_validation import StratifiedKFold

#Set the seed
random.seed(42)

In [90]:
# Open the file
file_open=open('train.csv')
# Open the csv reader over the file
csv_reader=csv.reader(file_open,delimiter=",")
# Read the first line which is the header
header=csv_reader.next()
# Load the dataset contained in the file
dataset=[]

cnt=0
for row in csv_reader:
#    if cnt<1:
#        print row
#        cnt+=1
    dataset.append(row)

#print header

In [91]:
# Replace the missing values
for row in dataset:
    row[header.index("VAR_0002")] = float(row[header.index("VAR_0002")]) if row[header.index("VAR_0002")] != "" else -1
#    row[header.index("VAR_0003")] = float(row[header.index("VAR_0003")]) if row[header.index("VAR_0003")] != "" else -1
#   row[header.index("VAR_0004")] = float(row[header.index("VAR_0004")]) if row[header.index("VAR_0004")] != "" else -1
#   row[header.index("VAR_0006")] = float(row[header.index("VAR_0006")]) if row[header.index("VAR_0006")] != "" else -1
#   row[header.index("VAR_0007")] = float(row[header.index("VAR_0007")]) if row[header.index("VAR_0007")] != "" else -1

In [92]:
# Filter the dataset based on the column name
feature_to_filter=["VAR_0002"]
indexes_to_filter=[]
for feature in feature_to_filter:
    indexes_to_filter.append(header.index(feature))

dataset_filtered=[]
for row in dataset:
    dataset_filtered.append([row[index] for index in indexes_to_filter])
# Build the structure containing the target
targets=[]
for row in dataset:
    targets.append(float(row[header.index("target")]))

IDVal = []
for row in dataset:
    IDVal.append(row[header.index("ID")])
    

In [93]:
#Split the datasets to have one for learning and the other for the test
train_dataset=[]
test_dataset=[]
train_target=[]
test_target=[]
test_ID=[]

for row,target in zip(dataset_filtered,targets):
    
    if random.random() < 0.1:
        train_dataset.append(row)
        train_target.append(target)
    else:
        test_dataset.append(row)
        test_target.append(target)

for row,target in zip(dataset_filtered,targets):
    
    full_dataset.append(row)
        
### Cross Validation ###

#cv = StratifiedKFold(train_dataset, n_folds=4)

###scoring
#clf = svm.SVC(kernel='linear', C=1)
clf=DecisionTreeClassifier()
scores = cross_validation.cross_val_score(clf, train_dataset, train_target, cv=5) 
print "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)

### getting the predictions ###
#predicted = cross_validation.cross_val_predict(clf, train_dataset, train_target, cv=10)
#print metrics.accuracy_score(train_target, predicted) 
clf.fit(train_dataset,train_target)
predictions=clf.predict(test_dataset)


Accuracy: 0.76 (+/- 0.00)


In [94]:
#Evaluate the quality of the prediction
#print sklearn.metrics.mean_absolute_error(predictions,test_target)

### AUC ###
#fpr, tpr, thresholds = sklearn.metrics.roc_curve(test_dataset, test_target, pos_label=2)
#print sklearn.metrics.roc_auc_score(fpr, tpr)

print roc_auc_score(test_target, predictions)





0.499070578838


In [99]:
#open('submission.csv', 'w') as csvfile:
#    fieldnames = ['ID', 'target']
#    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

import csv
from itertools import izip


# Open the file
file_open=open('test.csv')
# Open the csv reader over the file
csv_reader=csv.reader(file_open,delimiter=",")
# Read the first line which is the header
header=csv_reader.next()
# Load the dataset contained in the file
full_dataset=[]

for row in csv_reader:
    full_dataset.append(row)

ids=[]

for row in full_dataset:
    ids.append(row[header.index("ID")])
    
for row in full_dataset:
    row[header.index("VAR_0002")] = float(row[header.index("VAR_0002")]) if row[header.index("VAR_0002")] != "" else -1

# Filter the dataset based on the column name
feature_to_filter=["VAR_0002"]
indexes_to_filter=[]

for feature in feature_to_filter:
    indexes_to_filter.append(header.index(feature))

dataset_filtered=[]
for row in dataset:
    dataset_filtered.append([row[index] for index in indexes_to_filter])
predictionsOut=clf.predict(dataset_filtered)

with open('submission.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(["ID","target"])
    writer.writerows(izip(ids, predictionsOut))

#f = open("submission.csv", "w")
#f.write("ID","target")
#for i in xrange(len(test_target)):
#    f.write("{} {}\n".format(test_target[i], predictions[i]))

#f.close()
    